### Imports:


In [2]:
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR  # for building SVR model
from gradients import compute_face_phi,dphidx,dphidy,init

In [3]:
tec=np.genfromtxt("small_wave/tec.dat",skip_header=2).transpose()

# text='VARIABLES = X Y P U V u2 v2 w2 uv eps'
# Define variables from text-file
x = tec[0]
y = tec[1]
p = tec[2]
u = tec[3]
v = tec[4]
uu = tec[5]
vv = tec[6]
ww = tec[7]
uv = tec[8]
k_DNS = 0.5 * (uu + vv + ww)
eps_DNS = tec[9]



In [4]:
#Define matrix dimensions
if max(y) == 1.:
    ni = 170
    nj = 194
    nu = 1. / 10000.
else:
    nu = 1. / 10595.
    if max(x) > 8.:
        nj = 162
        ni = 162
    else:
        ni = 402
        nj = 162

viscos = nu

In [5]:
u2d = np.reshape(u, (nj, ni)).transpose()
v2d = np.reshape(v, (nj, ni)).transpose()
p2d = np.reshape(p, (nj, ni)).transpose()
x2d = np.reshape(x, (nj, ni)).transpose()
y2d = np.reshape(y, (nj, ni)).transpose()
uu2d = np.reshape(uu, (nj, ni)).transpose()  # =mean{v'_1v'_1}
uv2d = np.reshape(uv, (nj, ni)).transpose()  # =mean{v'_1v'_2}
vv2d = np.reshape(vv, (nj, ni)).transpose()  # =mean{v'_2v'_2}
ww2d = np.reshape(ww,(nj,ni)).transpose()
k2d = np.reshape(k_DNS, (nj, ni)).transpose()  # =mean{0.5(v'_iv'_i)}
eps_DNS2d = np.reshape(eps_DNS,(nj,ni)).transpose()

In [8]:
p2d[:, 1] = p2d[:, 2]
p2d[:, -1] = p2d[:, -1 - 1]

In [10]:
xc_yc = np.loadtxt("small_wave/xc_yc.dat")
xf = xc_yc[:, 0]
yf = xc_yc[:, 1]
xf2d = np.reshape(xf, (nj, ni)).transpose()
yf2d = np.reshape(yf, (nj, ni)).transpose()

In [12]:
xp2d = 0.25*(x2d[0:-1,0:-1]+x2d[0:-1,1:]+x2d[1:,0:-1]+x2d[1:,1:])
yp2d = 0.25*(y2d[0:-1,0:-1]+y2d[0:-1,1:]+y2d[1:,0:-1]+y2d[1:,1:])

In [15]:
x2d = np.delete(x2d, -1, 0)
y2d = np.delete(y2d, -1, 0)
xp2d = np.delete(xp2d, -1, 0)
yp2d = np.delete(yp2d, -1, 0)

# delete last columns
x2d = np.delete(x2d, -1, 1)
y2d = np.delete(y2d, -1, 1)
xp2d = np.delete(xp2d, -1, 1)
yp2d = np.delete(yp2d, -1, 1)

In [16]:
# compute geometric quantities
areaw, areawx, areawy, areas, areasx, areasy, vol, fx, fy = init(x2d, y2d, xp2d, yp2d)


In [18]:
# delete last row
u2d = np.delete(u2d, -1, 0)
v2d = np.delete(v2d, -1, 0)
p2d = np.delete(p2d, -1, 0)
k2d = np.delete(k2d, -1, 0)
uu2d = np.delete(uu2d, -1, 0)
vv2d = np.delete(vv2d, -1, 0)
ww2d = np.delete(ww2d, -1, 0)
uv2d = np.delete(uv2d, -1, 0)
eps_DNS2d = np.delete(eps_DNS2d, -1, 0)

# delete first row
u2d = np.delete(u2d, 0, 0)
v2d = np.delete(v2d, 0, 0)
p2d = np.delete(p2d, 0, 0)
k2d = np.delete(k2d, 0, 0)
uu2d = np.delete(uu2d, 0, 0)
vv2d = np.delete(vv2d, 0, 0)
ww2d = np.delete(ww2d, 0, 0)
uv2d = np.delete(uv2d, 0, 0)
eps_DNS2d = np.delete(eps_DNS2d, 0, 0)

# delete last columns
u2d = np.delete(u2d, -1, 1)
v2d = np.delete(v2d, -1, 1)
p2d = np.delete(p2d, -1, 1)
k2d = np.delete(k2d, -1, 1)
uu2d = np.delete(uu2d, -1, 1)
vv2d = np.delete(vv2d, -1, 1)
ww2d = np.delete(ww2d, -1, 1)
uv2d = np.delete(uv2d, -1, 1)
eps_DNS2d = np.delete(eps_DNS2d, -1, 1)

# delete first columns
u2d = np.delete(u2d, 0, 1)
v2d = np.delete(v2d, 0, 1)
p2d = np.delete(p2d, 0, 1)
k2d = np.delete(k2d, 0, 1)
uu2d = np.delete(uu2d, 0, 1)
vv2d = np.delete(vv2d, 0, 1)
ww2d = np.delete(ww2d, 0, 1)
uv2d = np.delete(uv2d, 0, 1)
eps_DNS2d = np.delete(eps_DNS2d, 0, 1)

### Set up dataframes for all simulations:

In [8]:
def dat_to_df (path1: str) -> pd.DataFrame:
    arr = np.genfromtxt(path1, skip_header=2).transpose()
    return pd.DataFrame({'x':arr[0],'y':arr[1],'p':arr[2],'u':arr[3],'v':arr[4],'uu':arr[5],'vv':arr[6],
                          'ww':arr[7],'uv':arr[8],'ε_dns':arr[9],'k_DNS':0.5*(arr[5]+arr[6]+arr[7])}) 


In [9]:
#test dataframe, test.dat contains a small sample from small_wave/tec.dat

df_test = dat_to_df("test.dat")
display(df_test)


,x,y,p,u,v,uu,vv,ww,uv,ε_dns,k_DNS
0,1.612685,-0.000568,-0.005847,0.017058,3.572080e-04,0.000049,2.578323e-08,0.000025,8.364178e-07,0.028104,0.000037
1,1.634877,-0.000164,-0.005784,0.016507,2.770773e-04,0.000047,1.894515e-08,0.000024,6.136777e-07,0.026793,0.000035
2,1.657070,0.000164,-0.005552,0.015912,2.194037e-04,0.000045,1.509294e-08,0.000023,4.549526e-07,0.025664,0.000034
3,1.679265,0.000419,-0.005404,0.015778,1.608201e-04,0.000045,1.231294e-08,0.000022,3.070192e-07,0.025285,0.000033
4,1.701461,0.000511,-0.005293,0.014699,2.288321e-07,0.000040,9.818966e-09,0.000020,-1.368973e-07,0.022976,0.000030
5,1.723659,0.000499,-0.004049,0.013955,9.820061e-06,0.000038,9.403140e-09,0.000020,-9.371006e-08,0.021911,0.000029
6,1.745856,0.000501,-0.003596,0.013766,6.878925e-06,0.000037,9.163382e-09,0.000019,-1.035498e-07,0.021406,0.000028
7,1.768054,0.000501,-0.002770,0.013436,3.650300e-06,0.000036,8.855914e-09,0.000019,-1.024379e-07,0.020896,0.000028
8,1.790253,0.000501,-0.002263,0.013319,4.192482e-06,0.000036,8.579529e-09,0.000019,-1.008055e-07,0.020521,0.000027
9,1.812451,0.000501,-0.001660,0.013155,2.548245e-06,0.000035,8.320124e-09,0.000018,-9.984931e-08,0.020182,0.000027


In [ ]:
df_tec_SW = dat_to_df("small_wave/tec.dat")
df_tec_SW = dat_to_df("large_wave/tec_large.dat")
df_tec_SW = dat_to_df("one_hill/tec_OneHill.dat")